In [ ]:
# define rooms and items
#Game Room
game_room = {
    "name": "game room",
    "type": "room",
}
piano = {
    "name": "piano",
    "type": "furniture",
}
couch = {
    "name": "couch",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}
key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

#BedRoom1
bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}
queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}
door_b = {
    "name": "door b",
    "type": "door",
}
key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}
door_c = {
    "name": "door c",
    "type": "door",
}
#BedRoom 2
bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}
double_bed = {
    "name": "double bed",
    "type": "furniture",
}
key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}
dresser = {
    "name": "dresser",
    "type": "furniture",
}


#Living Room

living_room = {
    "name": "living room",
    "type": "room",
}
dining_table = {
    "name": "dining table",
    "type": "furniture",
}
sink = {
    "name": "sink",
    "type": "accident",
}
door_d = {
    "name": "door d",
    "type": "door",
}
key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}
#Outside
outside = {
  "name": "outside"
}
all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [door_a, door_b, door_c, door_d]

all_furniture = [piano, queen_bed, double_bed, dresser, dining_table]

all_keys = [key_a, key_b, key_c, key_d]

all_accidents = [sink]

# define which items/rooms are related
# possibly this is not done. Will see when we run code

object_relations = {
    "game room": [couch, piano, door_a],
    "bedroom 1": [queen_bed, door_b, door_c],
    "bedroom 2": [double_bed, dresser, door_b],
    "outside": [door_d],
    "living room": [dining_table, sink, door_d, door_c],
    "door a": [game_room, bedroom_1], #probably should be keys as well assigned
    "door b": [bedroom_1, bedroom_2], #probably should be keys as well assigned
    "door c": [bedroom_1, living_room], #probably should be keys as well assigned
    "door d": [living_room, outside], #probably should be keys as well assigned
    "piano": [key_a],
    "queen bed": [key_b],
    "double bed": [key_c],
    "dresser": [key_d],
    "dining table": [],
    "couch": [],
    "sink": [],
}

# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "timer_running": False,
    "target_room": outside
}

In [ ]:
import threading
import time

stop_event = threading.Event()

def start_timer(seconds, stop_event):
    print(f"Timer started: {seconds} seconds remaining.")
    while seconds > 0 and not stop_event.is_set():
        minutes, secs = divmod(seconds, 60)
        timer = f"{minutes:02d}:{secs:02d}"
        print(timer, end="\r", flush=True)
        time.sleep(1)
        seconds -= 1

    if not stop_event.is_set():  # Timer expired
        print("\nTime's up! The room is now flooded. Life over.")
        exit()
        game_state["game_over"] = True  # Mark game as over
        stop_event.set()  # Trigger stop event to stop the game completely


NameError: name 'game_state' is not defined

In [ ]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state ["current_room"])

import threading


def play_room(room):
    """
    Play a room. Check if it's the target room or if the game is over.
    """
    global stop_event
    if game_state.get("game_over", False):  # Check if the game is over
        print("Game over! You can't continue.")
        return  # Exit the function to prevent further gameplay

    game_state["current_room"] = room
    if game_state["current_room"] == game_state["target_room"]:
        print("Congrats! You escaped the room!")
        stop_event.set()  # Stop the timer
        game_state["game_over"] = True  # End the game
        return  # Exit the function to prevent further gameplay

    while not game_state.get("game_over", False):  # Loop only if the game is not over
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        if game_state.get("game_over", False):  # Check after every input
            print("Game over! You can't continue.")
            return


        if intended_action == "explore":
            explore_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
        linebreak()


def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if room != current_room:
            return room
    return None


def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if item["name"] == item_name:
            # Handle accidents like sink flooding only for the sink
            if item["type"] == "accident" and item_name == "sink":
                if not game_state.get("timer_running", False):
                    print("You broke the sink and the room is starting to flood rapidly. You have only 2 minutes until the room is full of water and you won't be able to get out. RUN!!!")
                    specific_time = 120
                    game_state["timer_running"] = True
                    threading.Thread(target=start_timer, args=(specific_time, stop_event)).start()
                return play_room(current_room)

    for item in object_relations[current_room["name"]]:
        if item["name"] == item_name:
            output = "You examine " + item_name + ". "
            if (item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if (key["target"] == item):
                        have_key = True
                if have_key:
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if (item["name"] in object_relations and len(object_relations[item["name"]]) > 0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    if item["type"] == "furniture":
                        output += "There isn't anything interesting about it."
            print(output)
            break

    if output is None:
        print("The item you requested is not found in the current room.")

    if next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").strip() == 'yes':
        play_room(next_room)
    else:
        play_room(current_room)

In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()

NameError: name 'INIT_GAME_STATE' is not defined